In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_probability as tfp
import math as m

import matplotlib.pyplot as plt
import matplotlib.animation as animation

import os

In [2]:
lb = 1e-8
n_epoch = 2500
n_plot = 10
grad_max = 1e2
maxiters = 50000

lr_max = 5e-3
lr_min = 1e-5
lr_decay = 0.2
lr_decay_step = 500
w_decay = 1e-8

llb = lb
p_cutoff = -1.0
R = -1.0 / 8.314e-3

l_exp = np.arange(1,15)
n_exp = len(l_exp)

nr=8
ns=4

npara = nr * (ns + 4) + 1

l_train = []
l_val = []
for i in np.arange(0,n_exp):
    j = l_exp[i]
    if not (j in [2, 6, 9, 12]):
        l_train.append(i+1)
    else:
        l_val.append(i+1)

# 初始参数设定

In [3]:


llb = lb
p_cutoff = -1.0
R = -1.0 / 8.314e-3



# 实验数据读取

## 目录定义

In [4]:
expr_name = "4s8r-01"
os.chdir("..")
print("当前工作目录：%s"%(str(os.getcwd())))

# 不要重复执行这段代码

当前工作目录：w:\CRNN-TF2.0\CRNN-TF2.0


In [5]:
fig_path = str("./results/" + expr_name + "/figs")
ckpt_path = str("./results/" + expr_name + "/checkpoint")
script_path = str("./scripts")

if not os.path.exists(fig_path):
    os.makedirs(fig_path)

if not os.path.exists(ckpt_path):
    os.makedirs(ckpt_path)


# 数据导入

In [6]:
def load_exp(filename):
    exp_data = np.loadtxt(filename)
    
    # 归一化质量数据
    exp_data[:, 2] = exp_data[:, 2] / max(exp_data[:,2])
    return tf.constant(exp_data, dtype=tf.float32)

# 共十四个实验，依次编号为1，2，...，14
l_exp = np.arange(1,15)
n_exp = len(l_exp)

# 训练集与验证集的数字编号列表，其中[2,6,9,12]为测试集
l_train = []
l_val = []
for i in np.arange(1,n_exp+1):
    j = l_exp[i-1]
    if not (j in [2, 6, 9, 12]):
        l_train.append(i)
    else:
        l_val.append(i)

l_exp_data = []
l_exp_info = np.zeros(shape=(len(l_exp), 3))



In [7]:
for i_exp, value in enumerate(l_exp):
    filename = str("./exp_data/expdata_no"+str(value)+".txt")
    exp_data = load_exp(filename)
    if value == 4:
        exp_data = exp_data[:60, :]
    elif value == 5:
        exp_data = exp_data[:58, :]
    elif value == 6:
        exp_data = exp_data[:60, :]
    elif value == 7:
        exp_data = exp_data[:71, :]
    
    
    # Initial temperature
    l_exp_info[i_exp-1, 0] = exp_data[0, 1]

    exp_data = tf.constant(exp_data, dtype=tf.float32)

    l_exp_data.append(exp_data)



l_exp_info[:, 1] = np.loadtxt("exp_data/beta.txt")
l_exp_info[:, 2] = np.log(np.loadtxt("exp_data/ocen.txt")+llb)

l_exp_info = tf.constant(l_exp_info, dtype=tf.float32)

# 初始化p矩阵

In [8]:
p = np.random.randn(npara) * 1.e-2
p[:nr] += 0.8 # w_b
# nr~nr*(ns+1) vij
p[nr*(ns+1):nr*(ns+2)] += 0.8 # w_out
p[nr*(ns+2):nr*(ns+4)] += 0.1 # w_b | w_Ea
p[-1] = 0.1 # slope

p = tf.Variable(p, dtype=tf.float32)

# 定义CRNN方程

## 整理P矩阵

> 只能使用TensorFlow内部的张量操作，否则无法获得梯度。

In [9]:
@tf.function
def p2vec(p):
    slope = p[-1] * 1.e1
    
    w_b = p[:nr] * (slope * 10.0)
    w_b = tf.clip_by_value(w_b, clip_value_min=0., clip_value_max=50.)
    
    # julia 与 tf的reshape函数实现不同
    w_out = tf.reshape(p[nr:nr*(ns+1)],shape=(nr,ns))
    w_out = tf.transpose(w_out)

    temp1 = tf.clip_by_value(w_out[0:1, :], -3., 0.)
    temp2 = tf.clip_by_value(tf.abs(w_out[-1:, :]), 0., 3.)
    w_out = tf.concat([temp1, w_out[1:-1], temp2], axis=0)

    # 此段省略
    # julia原文：
    # if p_cutoff > 0.0
    #     w_out[findall(abs.(w_out) .< p_cutoff)] .= 0.0
    # end

    temp3 = -1*(tf.reduce_sum(w_out[:-2,:], 0) + w_out[-1:, :])
    temp3 = tf.reshape(temp3, shape=(1,nr))
    
    w_out = tf.concat([w_out[:-2,:], temp3, w_out[-1:,:]], axis=0)

    w_in_Ea = tf.abs(p[nr*(ns+1):nr*(ns+2)]) * slope * 100.
    w_in_Ea = tf.clip_by_value(w_in_Ea, 0., 300.)
    w_in_Ea = tf.reshape(w_in_Ea, shape=(1,8))

    w_in_b = tf.abs(p[nr*(ns+2):nr*(ns+3)])
    w_in_b = tf.reshape(w_in_b, shape=(1,8))

    w_in_ocen = tf.abs(p[nr*(ns+3):nr*(ns+4)])
    w_in_ocen = tf.clip_by_value(w_in_ocen, 0., 1.5)
    w_in_ocen = tf.reshape(w_in_ocen, shape=(1,8))

    # 此段省略
    # julia原文：
    # if p_cutoff > 0:
    #     w_in_ocean[abs(w_in_ocean)<p_cutoff] = 0.0
    
    w_in = tf.concat([
        tf.clip_by_value(w_out*(-1),0.,4.),
        w_in_Ea,
        w_in_b,
        w_in_ocen
    ], 0)

    return w_in, w_b, w_out

In [10]:
# 测试p
p_test = tf.constant(np.loadtxt("p.txt"), dtype=tf.float32)

## 定义CRNN结构

In [18]:
#@tf.function
#def getsampletemp(t, T0, beta):
#    t = tf.dtypes.cast(t, tf.float32)
#
#    T1 = T0 + t * (beta/60.0)
#    HR = tf.constant(40.0/60.0, dtype=tf.float32)
#    T2 = beta + 273.0
#    
#    temp1 = beta + 273.0 + HR * (t - 999.0*60.0)
#    temp2 = tf.constant(370.0+273.0, dtype=tf.float32)
#    T3 = tf.cond(tf.greater(temp1, temp2), lambda:temp2, lambda:temp1)
#
#    temp3 = 370.0+273.0 + HR * (t-1059.0*60.0)
#    temp4 = tf.constant(500.0+273.0, dtype=tf.float32)
#    T4 = tf.cond(tf.greater(temp3, temp4), lambda:temp4, lambda:temp3)
#    
#    T_branch1 = tf.cond(tf.greater(t, 1059. * 60.), lambda: T4, lambda: T3)
#    T_branch2 = tf.cond(tf.greater(t, 999. * 60.), lambda: T_branch1, lambda:T2)
#    T = tf.cond(tf.greater(beta, 100), lambda: T_branch2, lambda: T1)

#    return tf.reshape(T,shape=(1,1))

In [30]:
global T0
global beta
global ocen
#global l_exp_info

#@tf.function
def crnn(t, u):
    logX = tf.dtypes.cast(tf.math.log(tf.reshape(tf.clip_by_value(u, lb, 10.), shape=(4,1))),tf.float32)
    
    # getsampletemp moved to here

    T1 = T0 + t * (beta/60.0)
    HR = tf.constant(40.0/60.0, dtype=tf.float32)
    T2 = beta + 273.0
    
    temp1 = beta + 273.0 + HR * (t - 999.0*60.0)
    temp2 = tf.constant(370.0+273.0, dtype=tf.float32)
    T3 = tf.cond(tf.greater(temp1, temp2), lambda:temp2, lambda:temp1)

    temp3 = 370.0+273.0 + HR * (t-1059.0*60.0)
    temp4 = tf.constant(500.0+273.0, dtype=tf.float32)
    T4 = tf.cond(tf.greater(temp3, temp4), lambda:temp4, lambda:temp3)
    
    T_branch1 = tf.cond(tf.greater(t, 1059. * 60.), lambda: T4, lambda: T3)
    T_branch2 = tf.cond(tf.greater(t, 999. * 60.), lambda: T_branch1, lambda:T2)
    T = tf.cond(tf.greater(beta, 100), lambda: T_branch2, lambda: T1)

    T = tf.reshape(T, shape=(1,1))
    ####################################

    
    #global p
    w_in, w_b, w_out = p2vec(p)
    w_in_X = tf.matmul(tf.transpose(w_in), tf.concat([logX, R/T, tf.math.log(T), ocen],0))
    du_dt = tf.matmul(w_out, tf.exp(tf.reshape(w_in_X, shape=(nr,1))+tf.reshape(w_b, shape=(nr,1))))
    return du_dt

u0 = np.zeros(ns)
u0[0] = 1
u0 = tf.constant(u0, shape=(ns,),dtype=tf.float32)

@tf.function
def pred_n_ode(p, i_exp, exp_data):
    global T0
    global beta
    global ocen
    T0 = l_exp_info[i_exp-1:i_exp, 0:1]
    beta = l_exp_info[i_exp-1:i_exp, 1:2]
    ocen = l_exp_info[i_exp-1:i_exp, 2:]
    
    ts = exp_data[:, 0]
    
    tspan = [ts[0],ts[-1]]

    solution = tf.function(lambda: tfp.math.ode.BDF().solve(
        crnn,
        tspan[0],#t_init
        u0,
        solution_times=ts
    ), autograph=True)()
    return solution.states


In [22]:
u_test = pred_n_ode(p_test, 1, exp_data)

In [31]:
# pred_n_ode测试
p_test = tf.Variable(p_test, dtype=tf.float32)
with tf.GradientTape() as tape:
    tape.watch(p_test)
    pred_test = pred_n_ode(p_test, 1, exp_data) 

grad = tape.gradient(pred_test, p_test)    


In [32]:
print(grad)

None


# 定义损失函数

In [33]:
@tf.function
def loss_n_ode(p, i_exp):
    global ocen
    exp_data = l_exp_data[i_exp-1]
    pred = pred_n_ode(p, i_exp, exp_data)
    
    masslist = tf.reduce_sum(tf.clip_by_value(pred[:-1,:], 0, 1e3),axis=0)
    gaslist = tf.clip_by_value(pred[-1:,:], 0, 1e3)

    loss = tf.keras.losses.MAE(masslist, exp_data[:len(masslist),2])
    
    loss = tf.cond(tf.greater(ocen, 1000.0), lambda: loss, lambda:loss + tf.keras.losses.MAE(gaslist, (1-exp_data[:len(masslist),2])))
    
    return loss

In [34]:
loss_n_ode(p_test,1)

InaccessibleTensorError: in user code:

    File "<ipython-input-33-6a68311360d6>", line 12, in loss_n_ode  *
        loss = tf.cond(tf.greater(ocen, 1000.0), lambda: loss, lambda:loss + tf.keras.losses.MAE(gaslist, (1-exp_data[:len(masslist),2])))

    InaccessibleTensorError: <tf.Tensor 'strided_slice_2:0' shape=(1, 1) dtype=float32> is out of scope and cannot be used here. Use return values, explicit Python locals or TensorFlow collections to access it.
    Please see https://www.tensorflow.org/guide/function#all_outputs_of_a_tffunction_must_be_return_values for more information.
    
    <tf.Tensor 'strided_slice_2:0' shape=(1, 1) dtype=float32> was defined here:
        File "c:\Program Files\Python37\lib\runpy.py", line 193, in _run_module_as_main
          "__main__", mod_spec)
        File "c:\Program Files\Python37\lib\runpy.py", line 85, in _run_code
          exec(code, run_globals)
        File "C:\Users\marcl\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher

# 训练过程

In [ ]:
opt = ...
for epoch in range(n_epoch):
    print("Start epoch %d" %(epoch))
    
    for i_exp in (shuffled index of experiments):
        if i_exp in l_val:
            continue

        with tf.GradientTape() as tape:
            tape.watch(p)
            loss = loss_n_ode(p, i_exp)
        
        grad = tape.gradient(loss, p)
        opt.apply_gradients(zip([grad],[p]))
    


In [ ]:
# 这个不行
@tf.function
def 波波人(t, u):
    if t>10.0:
        du_dt = u*t
    else:
        du_dt = 2*u*t
    return du_dt

# 这个可以
@tf.function
def 波波人(t, u):
    du_dt = tf.cond(tf.greater(t, 10.0), lambda: u*t, lambda:2*u*t)
    return du_dt